# 1. Imports

In [1]:
# data processing
import pandas as pd

# web/xml scraping
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# local tests
import zipfile
import io
import os
import re

source_url = 'https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesContratante.aspx'

# 2. Explore

In [2]:
# Helper functions
def get_soup(url):
    response = requests.get(url)

    return BeautifulSoup(response.text, 'html.parser')

def recursive_field_dict(field, field_dict):
    for child in field:
        tag = child.tag.split('}')[-1]
        if len(child) == 0:
            field_dict[tag] = child.text
        else:
            if tag not in field_dict:
                field_dict[tag] = {}
                recursive_field_dict(child, field_dict[tag])

def recursive_find_value(tag, dictionary):
    if tag in dictionary:
        return tag, dictionary[tag]
    else:
        for key, value in dictionary.items():
            if isinstance(value, dict):
                result = recursive_find_value(tag, value)
                if result:
                    return result
                else:
                    pass

def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif v:
            items.append((new_key, v))
    return dict(items)

def get_atom_data(file):
    tree = ET.parse(file)
    root = tree.getroot()
    ns = {node[0]: node[1] for _, node in ET.iterparse(file, events=['start-ns'])}
    atom = "{" + ns[''] + "}"

    for k, v in ns.items():
        try:
            ET.register_namespace(k, v)
        except ValueError:
            pass

    entries = root.findall(f'{atom}entry')

    return ns, atom, entries

# Get the soup of the source data url
soup = get_soup(source_url)

# Create a dict with keys the period of the data and the link to the data as values
links = [a.get('href') for a in soup.find_all('a') if 'contratacion' in a.get('href') and a.get('href').endswith('zip')]
periods = [int(link[113:-4]) for link in links]
source_data = {p: l for p, l in zip(periods, links)}

In [3]:
# One file for testing purposes: jan2025
test = 202501
# test_zip_url = source_data[test]

In [ ]:
# Get the data of test period
response = requests.get(test_zip_url)

with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
    thezip.extractall(f'data/{test}')

In [47]:
len(test_files)*500*12

564000

In [4]:
# open one file for testing
test_files = os.listdir(f'data/{test}')
test_file = os.path.join(f'data/{test}', test_files[0])

ns, atom, entries = get_atom_data(test_file)

In [5]:
data = []

for entry in entries:
    entry_data = {}

    # Extract general information
    for field in entry:
        tag = field.tag.split('}')[-1]
        entry_data[tag] = field.text if tag != 'link' else field.get('href')

    # Extract summary information
    summary = entry_data['summary']
    entry_data['contracting_entity'] = re.search(r'Órgano de Contratación: (.*?);', summary).group(1)
    entry_data['amount'] = re.search(r'Importe: (.*?);', summary).group(1).split()[0]
    entry_data['currency'] = re.search(r'Importe: (.*?);', summary).group(1).split()[1]

    # Remove summary column from entry_data
    entry_data.pop('summary')

    # Extract details information
    details = entry.find('cac-place-ext:ContractFolderStatus', ns)
    entry_data['id'] = details.find('cbc:ContractFolderID', ns).text
    entry_data['status'] = details.find('cbc-place-ext:ContractFolderStatusCode', ns).text

    data.append(entry_data)

df = pd.DataFrame(data)


In [50]:
print(df.link.iloc[0])

https://contrataciondelestado.es/wps/poc?uri=deeplink:detalle_licitacion&idEvl=jJX4vUsoko5%2FR5QFTlaM4A%3D%3D


In [6]:
df.head()

,id,link,title,updated,ContractFolderStatus,contracting_entity,amount,currency,status
0,2/2024,https://contrataciondelestado.es/wps/poc?uri=d...,Acuerdo Marco para la Contratación del Suminis...,2025-01-21T09:26:00.916+01:00,\n,Secretaría General de la Federación de Municip...,0,EUR,PUB
1,300/2024/00986,https://contrataciondelestado.es/wps/poc?uri=d...,Suministro de Alimentos para la Residencia Int...,2025-01-21T09:25:48.150+01:00,\n,"Área de Gobierno de Políticas Sociales, Famili...",129462.76,EUR,EV
2,ACIISI-2405,https://contrataciondelestado.es/wps/poc?uri=d...,Suministro de equipos enrutadores IP para los ...,2025-01-21T09:25:46.667+01:00,\n,"Consejería de Universidades, Ciencia e Innovac...",690200,EUR,EV
3,534/2024,https://contrataciondelestado.es/wps/poc?uri=d...,"Reordenación Depuración Guía-Gáldar, impulsión...",2025-01-21T09:25:45.767+01:00,\n,Vicepresidencia del Consejo Insular de Aguas d...,1768598.73,EUR,EV
4,335/2023 PROCEDIMIENTO ADJUDICACION NOVIEMBRE ...,https://contrataciondelestado.es/wps/poc?uri=d...,Concesión de 10 plazas de garaje en residencia...,2025-01-21T09:25:42.565+01:00,\n,Junta de Gobierno Local del Ayuntamiento de La...,0,EUR,PUB


In [7]:
data = []

for entry in entries:
    # Initialize entry data
    entry_data = {}

    # Extract general information
    for field in entry:
        tag = field.tag.split('}')[-1]
        entry_data[tag] = field.text if tag != 'link' else field.get('href')

    # Generate full details information
    details = entry.find('cac-place-ext:ContractFolderStatus', ns)
    details_dict = {}
    recursive_field_dict(details, details_dict)
    flat_details = flatten_dict(details_dict)

    # Add specific information to entry data


    data.append(entry_data)

df = pd.DataFrame(data)

In [10]:
df.head()

,id,link,summary,title,updated,ContractFolderStatus
0,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,Id licitación: 2/2024; Órgano de Contratación:...,Acuerdo Marco para la Contratación del Suminis...,2025-01-21T09:26:00.916+01:00,\n
1,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,Id licitación: 300/2024/00986; Órgano de Contr...,Suministro de Alimentos para la Residencia Int...,2025-01-21T09:25:48.150+01:00,\n
2,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,Id licitación: ACIISI-2405; Órgano de Contrata...,Suministro de equipos enrutadores IP para los ...,2025-01-21T09:25:46.667+01:00,\n
3,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,Id licitación: 534/2024; Órgano de Contratació...,"Reordenación Depuración Guía-Gáldar, impulsión...",2025-01-21T09:25:45.767+01:00,\n
4,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,Id licitación: 335/2023 PROCEDIMIENTO ADJUDICA...,Concesión de 10 plazas de garaje en residencia...,2025-01-21T09:25:42.565+01:00,\n


In [53]:
no_null_cols = [col for col in df_test.columns if df_test[col].isnull().sum() == 0]
df_no_nulls = df_test[no_null_cols]

In [ ]:
main_info_dict = {
    'id': 'ContractFolderID',
    'cp_name': 'LocatedContractingParty.Party.PartyName.Name',
    'cp_id': 'LocatedContractingParty.Party.PartyIdentification.ID',
    'cp_city': 'LocatedContractingParty.Party.PostalAddress.CityName',
    'cp_zip_code': 'LocatedContractingParty.Party.PostalAddress.PostalZone',
    'cp_country': 'LocatedContractingParty.Party.PostalAddress.Country.IdentificationCode',
    'cp_contact_email': 'LocatedContractingParty.Party.Contact.ElectronicMail',
    'status': 'ContractFolderStatusCode',
    'pp_name': 'ProcurementProject.ProjectName',
    'pp_type_code': 'ProcurementProject.ProjectTypeCode',
    'tender_funding': 'TenderingTerms.FundingProgramCode',
    'pp_budget_amount_tax_exclusive': 'ProcurementProject.BudgetAmount.TaxExclusiveAmount',
    'pp_budget_amount_estimated_overall_contract_amount': 'ProcurementProject.BudgetAmount.EstimatedOverallContractAmount'
}

In [57]:
df_no_nulls['TenderingTerms.FundingProgramCode']

0      NO-EU
1      NO-EU
2      NO-EU
3      NO-EU
4      NO-EU
       ...  
495    NO-EU
496    NO-EU
497     PRTR
498    NO-EU
499    NO-EU
Name: TenderingTerms.FundingProgramCode, Length: 500, dtype: object

In [ ]:
df_no_nulls.columns

Index(['ContractFolderID', 'ContractFolderStatusCode',
       'LocatedContractingParty.ContractingPartyTypeCode',
       'LocatedContractingParty.ActivityCode',
       'LocatedContractingParty.BuyerProfileURIID',
       'LocatedContractingParty.Party.PartyIdentification.ID',
       'LocatedContractingParty.Party.PartyName.Name',
       'LocatedContractingParty.Party.PostalAddress.CityName',
       'LocatedContractingParty.Party.PostalAddress.PostalZone',
       'LocatedContractingParty.Party.PostalAddress.AddressLine.Line',
       'LocatedContractingParty.Party.PostalAddress.Country.IdentificationCode',
       'LocatedContractingParty.Party.PostalAddress.Country.Name',
       'LocatedContractingParty.Party.Contact.Name',
       'LocatedContractingParty.Party.Contact.ElectronicMail',
       'LocatedContractingParty.ParentLocatedParty.PartyName.Name',
       'LocatedContractingParty.ParentLocatedParty.ParentLocatedParty.PartyName.Name',
       'LocatedContractingParty.ParentLocatedParty.

In [42]:
for col in df_test.columns:
    print(col)

ContractFolderID
ContractFolderStatusCode
UUID
LocatedContractingParty.ContractingPartyTypeCode
LocatedContractingParty.ActivityCode
LocatedContractingParty.BuyerProfileURIID
LocatedContractingParty.Party.WebsiteURI
LocatedContractingParty.Party.PartyIdentification.ID
LocatedContractingParty.Party.PartyName.Name
LocatedContractingParty.Party.PostalAddress.CityName
LocatedContractingParty.Party.PostalAddress.PostalZone
LocatedContractingParty.Party.PostalAddress.AddressLine.Line
LocatedContractingParty.Party.PostalAddress.Country.IdentificationCode
LocatedContractingParty.Party.PostalAddress.Country.Name
LocatedContractingParty.Party.Contact.Name
LocatedContractingParty.Party.Contact.Telephone
LocatedContractingParty.Party.Contact.ElectronicMail
LocatedContractingParty.ParentLocatedParty.PartyName.Name
LocatedContractingParty.ParentLocatedParty.ParentLocatedParty.PartyName.Name
LocatedContractingParty.ParentLocatedParty.ParentLocatedParty.ParentLocatedParty.PartyName.Name
LocatedContrac